In [ ]:
import numpy as np
import tifffile
import colorsys
import random
from pathlib import Path
import matplotlib.patches as patches
import matplotlib.patheffects as PathEffects
from matplotlib.patches import ConnectionPatch
from matplotlib.transforms import Bbox
import xml.etree.ElementTree as ET
import importlib
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from multiplex2brightfield import converter, convert_from_file, configuration_presets



In [ ]:
def read_and_visualize_multiplex(filepath, target_channel_names, brightness=0.5, level=1):
    """
    Reads a multiplex image and creates an RGB visualization where each channel 
    has a distinct color, matching channels from the file metadata.
    
    Parameters:
    -----------
    filepath : Path or str
        Path to the OME-TIFF file
    target_channel_names : list
        List of channel names to visualize
    brightness : float
        Overall brightness factor
    level : int
        Pyramid level to read (1 is full resolution, lower numbers are higher resolution if available,
        higher numbers are lower resolution)
    """
    random.seed(7)

    # Read the image and metadata
    with tifffile.TiffFile(filepath) as tif:
        # Get metadata for channel names and pixel size information
        ome_metadata = tif.ome_metadata
        metadata = tif.pages[0].tags['ImageDescription'].value

        # Parse the XML metadata
        root = ET.fromstring(metadata)
        ns_uri = root.tag.split('}')[0].strip('{')
        ns = {'ome': ns_uri}

        # Extract channel names from metadata
        channel_elements = root.findall('.//ome:Channel', ns)
        file_channel_names = [channel.get('Name') for channel in channel_elements if channel.get('Name')]

        # Determine the number of available pyramid levels and adjust level index (zero-based)
        n_levels = len(tif.series[0].levels)
        level_index = level - 1
        print(f"Available pyramid levels: {n_levels} (levels 1 to {n_levels})")
        print(f"Requested level: {level}")
        if level_index >= n_levels:
            print(f"Warning: Requested level {level} not available. Using highest available level {n_levels}")
            level_index = n_levels - 1
        elif level_index < 0:
            print(f"Warning: Requested level {level} not available. Using lowest available level 1")
            level_index = 0

        # Compute effective pixel size at the requested pyramid level
        pixels = root.find('.//ome:Pixels', ns)
        if pixels is not None:
            phys_size_x = pixels.get('PhysicalSizeX')
            phys_size_y = pixels.get('PhysicalSizeY')
            phys_size_x_unit = pixels.get('PhysicalSizeXUnit', '')
            phys_size_y_unit = pixels.get('PhysicalSizeYUnit', '')
            try:
                phys_size_x = float(phys_size_x)
                phys_size_y = float(phys_size_y)
            except (ValueError, TypeError):
                phys_size_x = None
                phys_size_y = None

            # Get the shape of the full resolution image (assumed level 1) and the chosen pyramid level.
            base_shape = tif.series[0].levels[0].shape  # full resolution
            current_shape = tif.series[0].levels[level_index].shape

            # Assuming the last two dimensions are (height, width), compute the scaling factors.
            scale_x = base_shape[-1] / current_shape[-1]
            scale_y = base_shape[-2] / current_shape[-2]

            if phys_size_x is not None and phys_size_y is not None:
                effective_phys_size_x = phys_size_x * scale_x
                effective_phys_size_y = phys_size_y * scale_y
                print("Effective pixel size at pyramid level {}:".format(level))
                print("  PhysicalSizeX: {} {}".format(effective_phys_size_x, phys_size_x_unit))
                print("  PhysicalSizeY: {} {}".format(effective_phys_size_y, phys_size_y_unit))
            else:
                print("Pixel size information not found or invalid in metadata.")
        else:
            print("Pixel size information not found in metadata.")

        # Read the requested pyramid level
        img = tif.series[0].levels[level_index].asarray()
        print(f"Image shape at level {level}: {img.shape}")
        if len(img.shape) < 4:
            img = np.expand_dims(img, axis=0)  # Add z-axis if missing.
            print("No z level found. Adding one.")
        print("Channels found in file:", file_channel_names)
        print("Number of channels in data:", img.shape[0])
        print("Number of channels in metadata:", len(file_channel_names))
        
    print("Final image shape:", img.shape)
    
    # Create RGB visualization
    rgb_img_stack = []
    for z in range(img.shape[0]):
        rgb_img = np.zeros((*img.shape[2:], 3), dtype=np.float32)
        base_colors = [
            (255, 0, 0),      # red
            (0, 255, 0),      # green
            (0, 0, 255),      # blue
            (255, 255, 0),    # yellow
            (255, 0, 255),    # magenta
            (0, 255, 255),    # cyan
            (128, 0, 128),    # bop purple
            (255, 165, 0),    # bop orange
            (31, 168, 241),   # bop blue
        ]
        clean_file_names = [name.replace('-', '_').lower() for name in file_channel_names]
        channel_mapping = {name: idx for idx, name in enumerate(clean_file_names)}

        for i, channel_name in enumerate(target_channel_names):
            clean_name = channel_name.replace('-', '_').lower()
            if clean_name in channel_mapping:
                idx = channel_mapping[clean_name]
                color = (np.array(base_colors[i]) / 255.0 if i < len(base_colors) 
                         else np.array(colorsys.hsv_to_rgb(random.uniform(0, 1), 1, 1)))
                channel_data = img[z, idx].astype(float)
                if channel_data.max() > channel_data.min():
                    channel_data = (channel_data - channel_data.min()) / (channel_data.max() - channel_data.min())
                for c in range(3):
                    rgb_img[..., c] += brightness * len(target_channel_names) * channel_data * color[c]

        rgb_img = np.clip(rgb_img, 0, 1)
        rgb_img_stack.append(rgb_img)
    
    if img.shape[0] == 1:
        return rgb_img
    else:
        return np.array(rgb_img_stack)


In [ ]:


def add_scalebar(ax, pixel_size, scalebar_length, length_unit='mm',
                 bar_color='white', text_color='white',
                 bar_thickness_pt=3, margin_pt=10,
                 manual_bar_thickness_pt=None, text_offset_multiplier=1.75,
                 show_text=True, font_size=10):
    """
    Adds a scale bar with fixed screen-space offsets and an optional text label whose vertical
    offset is proportional to the actual bar thickness. If a manual thickness is provided,
    that thickness is used for both the bar and for computing the text offset.
    
    Parameters:
      ax                      : The matplotlib Axes to draw on.
      pixel_size              : Image pixel size in µm.
      scalebar_length         : Desired physical length of the scale bar.
      length_unit             : 'mm' or 'um' (case‐insensitive).
      bar_color               : Color of the scale bar.
      text_color              : Color of the label text.
      bar_thickness_pt        : Base bar thickness in points (will be halved if not overridden).
      margin_pt               : Margin in points from the axes edge (the effective margin is half this value).
      manual_bar_thickness_pt : Optional manual thickness (in points) for the bar.
      text_offset_multiplier  : Factor to multiply the bar thickness to determine the vertical offset
                                for the label (e.g. 1.0 places the text one line thickness above the bar).
      show_text               : Boolean flag; if False, only the scale bar is drawn.
      font_size               : Font size for the scale bar label text.
    """
    # Convert scalebar_length to micrometers and create the label.
    if length_unit.lower() == 'mm':
        length_um = scalebar_length * 1000
        label = f"{scalebar_length} mm"
    elif length_unit.lower() in ['um', 'µm']:
        length_um = scalebar_length
        label = f"{scalebar_length} µm"
    else:
        raise ValueError("Unknown length unit; use 'mm' or 'um'.")
    
    # Get image dimensions (in pixels) from the first image on the axis.
    im = ax.get_images()[0]
    img_data = im.get_array()
    height_px, width_px = img_data.shape[:2]
    
    # Calculate desired scale bar length in image pixels.
    scale_bar_px = length_um / pixel_size  # (µm) / (µm/pixel)
    # Determine fraction of the axis width that this bar occupies.
    bar_width_frac = scale_bar_px / width_px

    # Get the axes' bounding box in display (pixel) coordinates.
    renderer = ax.figure.canvas.get_renderer()
    bbox = ax.get_window_extent(renderer=renderer)
    axis_width_pixels = bbox.width
    axis_height_pixels = bbox.height

    # Convert fixed offsets from points to pixels.
    margin_pixels = margin_pt * ax.figure.dpi / 72.0
    # Move the bar/text halfway toward the corner.
    effective_margin = margin_pixels / 2.0

    # Use the manual thickness if provided; otherwise, use half of bar_thickness_pt.
    if manual_bar_thickness_pt is not None:
        bar_thickness_pixels = manual_bar_thickness_pt * ax.figure.dpi / 72.0
    else:
        bar_thickness_pixels = (bar_thickness_pt / 2) * ax.figure.dpi / 72.0

    # Set the text vertical offset proportional to the bar thickness.
    text_offset_pixels = bar_thickness_pixels * text_offset_multiplier

    # Compute the bar's width in display pixels.
    bar_width_pixels = bar_width_frac * axis_width_pixels

    # Determine the lower-left corner of the bar in display space.
    bar_bl_disp = (bbox.x0 + effective_margin, bbox.y0 + effective_margin)
    # Upper-right corner in display space:
    bar_tr_disp = (bbox.x0 + effective_margin + bar_width_pixels,
                   bbox.y0 + effective_margin + bar_thickness_pixels)
    
    # Convert these display coordinates back to normalized (axes) coordinates.
    bl_axes = ax.transAxes.inverted().transform(bar_bl_disp)
    tr_axes = ax.transAxes.inverted().transform(bar_tr_disp)
    rect_width = tr_axes[0] - bl_axes[0]
    rect_height = tr_axes[1] - bl_axes[1]
    
    # Draw the scale bar.
    rect = patches.Rectangle(bl_axes, rect_width, rect_height,
                             transform=ax.transAxes,
                             color=bar_color,
                             clip_on=False)
    ax.add_patch(rect)
    
    # If requested, add the label text.
    if show_text:
        text_disp = (bbox.x0 + effective_margin + bar_width_pixels / 2,
                     bbox.y0 + effective_margin + bar_thickness_pixels + text_offset_pixels)
        text_axes = ax.transAxes.inverted().transform(text_disp)
        txt = ax.text(text_axes[0], text_axes[1], label,
                      transform=ax.transAxes,
                      color=text_color,
                      ha='center', va='bottom', fontsize=font_size)
        # Add a semi-transparent black shadow behind the text.
        txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground=(0, 0, 0, 0.5))])




In [ ]:

input_filename = "high-parametric-multiplexed-immunofluorescence/8023452/2022_08_9_Tonsil_IF_PositiveIF.ome.tiff"

output_filename = "output/2022_08_9_Tonsil_IF_PositiveIF_H&E.ome.tiff"

config = configuration_presets.GetConfiguration('Axioscan 7')

config["components"]["haematoxylin"]["color"].update({"R": 0, "G": 24, "B": 150})
config["components"]["eosinophilic"]["color"].update({"R": 188, "G": 128, "B": 165})
config["components"]["epithelial"]["color"].update({"R": 182, "G": 170, "B": 183})
config["components"]["erythrocytes"]["color"].update({"R": 180, "G": 60, "B": 103})
config["background"]["color"].update({"R": 203, "G": 192, "B": 196})

config["components"]["haematoxylin"]["intensity"] = 2
config["components"]["eosinophilic"]["intensity"] = 0.5
config["components"]["epithelial"]["intensity"] = 1.5
config["components"]["erythrocytes"]["intensity"] = 1.0

config["components"]["haematoxylin"]["median_filter_size"] = 0
config["components"]["haematoxylin"]["gaussian_filter_sigma"] = 0
config["components"]["haematoxylin"]["sharpen_filter_radius"] = 2
config["components"]["haematoxylin"]["sharpen_filter_amount"] = 2
config["components"]["haematoxylin"]["normalize_percentage_min"] = 60
config["components"]["haematoxylin"]["normalize_percentage_max"] = 90

config["components"]["eosinophilic"]["median_filter_size"] = 0
config["components"]["eosinophilic"]["gaussian_filter_sigma"] = 0
config["components"]["eosinophilic"]["sharpen_filter_radius"] = 2
config["components"]["eosinophilic"]["sharpen_filter_amount"] = 2
config["components"]["eosinophilic"]["normalize_percentage_min"] = 30
config["components"]["eosinophilic"]["normalize_percentage_max"] = 90

config["components"]["epithelial"]["median_filter_size"] = 0
config["components"]["epithelial"]["gaussian_filter_sigma"] = 0
config["components"]["epithelial"]["sharpen_filter_radius"] = 2
config["components"]["epithelial"]["sharpen_filter_amount"] = 2
config["components"]["epithelial"]["normalize_percentage_min"] = 0
config["components"]["epithelial"]["normalize_percentage_max"] = 70

config["components"]["erythrocytes"]["median_filter_size"] = 0
config["components"]["erythrocytes"]["gaussian_filter_sigma"] = 0
config["components"]["erythrocytes"]["sharpen_filter_radius"] = 0
config["components"]["erythrocytes"]["sharpen_filter_amount"] = 0
config["components"]["erythrocytes"]["normalize_percentage_min"] = 80
config["components"]["erythrocytes"]["normalize_percentage_max"] = 99

config["components"]["haematoxylin"].update({"targets": ['DAPI']})
config["components"]["eosinophilic"].update({"targets": ['Cy5']})
config["components"]["epithelial"].update({"targets": ['AF488']})
config["components"]["erythrocytes"].update({"targets": ['AF555']})

config["components"]["haematoxylin"].update({"histogram_normalisation": False})
config["components"]["haematoxylin"].update({"clip": [0.1, 1]})
config["components"]["erythrocytes"].update({"clip": [0.2, 1]})


del config["components"]["marker"]


result = convert_from_file(
    input_filename,
    output_filename,
    output_pixel_size_x = 0.3441,
    output_pixel_size_y = 0.3441,
    config = config
)

image_HE_IF = np.transpose(result[0], (1,2,0))  # Remove Z dimension and reorder to (H,W,C)

plt.figure(figsize=(10,10))
plt.imshow(image_HE_IF)
plt.axis('off')
plt.title(f'Processed Image')
plt.show()

In [ ]:

file_path = Path("high-parametric-multiplexed-immunofluorescence/8023452/2022_08_9_Tonsil_IF_PositiveIF.ome.tiff")
target_channels = ['DAPI', 'AF555', 'Cy5','AF488']

image_IF = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.5, level=1)

image_HE = tifffile.imread("high-parametric-multiplexed-immunofluorescence/8023452/2022_08_10_Tonsil_IF_HE_PostiveIF.ome.tif", level=1)

print(image_IF.shape)
print(image_HE_IF.shape)
print(image_HE.shape)

# Create figure with two subplots side by side
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 8))

# Display images
im1 = ax1.imshow(image_IF[750:-350, 400:-250, :])
im2 = ax2.imshow(image_HE_IF[750:-350, 400:-250, :])
im3 = ax3.imshow(image_HE)

# Set titles if provided
ax1.set_title("IF")
ax2.set_title("Virtual H&E from IF")
ax3.set_title("Real H&E")

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


In [ ]:

input_filename = "high-parametric-multiplexed-immunofluorescence/8023452/RNAPos Tonsil_20220815-162517-674186_Q001.ome.tif"
output_filename = "output/RNAPos Tonsil_20220815-162517-674186_Q001_H&E.ome.tif"

config = configuration_presets.GetConfiguration('Axioscan 7')

config["components"]["haematoxylin"]["color"].update({"R": 0, "G": 24, "B": 150})
config["components"]["epithelial"]["color"].update({"R": 188, "G": 128, "B": 165})
config["components"]["eosinophilic"]["color"].update({"R": 182, "G": 170, "B": 183})
config["background"]["color"].update({"R": 203, "G": 192, "B": 196})

config["components"]["haematoxylin"]["intensity"] = 3
config["components"]["eosinophilic"]["intensity"] = 5.0
config["components"]["epithelial"]["intensity"] = 1.0

# config["components"]["haematoxylin"]["median_filter_size"] = 0
# config["components"]["haematoxylin"]["gaussian_filter_sigma"] = 0
# config["components"]["haematoxylin"]["sharpen_filter_radius"] = 2
# config["components"]["haematoxylin"]["sharpen_filter_amount"] = 2
config["components"]["haematoxylin"]["normalize_percentage_min"] = 70
config["components"]["haematoxylin"]["normalize_percentage_max"] = 90

# config["components"]["eosinophilic"]["median_filter_size"] = 0
# config["components"]["eosinophilic"]["gaussian_filter_sigma"] = 0
# config["components"]["eosinophilic"]["sharpen_filter_radius"] = 2
# config["components"]["eosinophilic"]["sharpen_filter_amount"] = 2
config["components"]["eosinophilic"]["normalize_percentage_min"] = 40
config["components"]["eosinophilic"]["normalize_percentage_max"] = 90

# config["components"]["epithelial"]["median_filter_size"] = 0
# config["components"]["epithelial"]["gaussian_filter_sigma"] = 0
# config["components"]["epithelial"]["sharpen_filter_radius"] = 2
# config["components"]["epithelial"]["sharpen_filter_amount"] = 2
config["components"]["epithelial"]["normalize_percentage_min"] = 10
config["components"]["epithelial"]["normalize_percentage_max"] = 90

# config["components"]["haematoxylin"].update({"histogram_normalisation": True})
# config["components"]["haematoxylin"].update({"clip": [0, 1]})

del config["components"]["marker"]
# del config["components"]["haematoxylin"]
# del config["components"]["epithelial"]
# del config["components"]["eosinophilic"]
# del config["components"]["erythrocytes"]



config["components"]["haematoxylin"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["eosinophilic"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["epithelial"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["erythrocytes"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})

result_no_filter = convert_from_file(input_filename,
        output_filename = "output/RNAPos Tonsil_20220815-162517-674186_Q001_H&E_no_filter.ome.tif",
        # AI_enhancement = False,
        output_pixel_size_x = 0.1721,
        output_pixel_size_y = 0.1721,
        config = config
        )

config["components"]["haematoxylin"].update({"gaussian_filter_sigma": 0, "median_filter_size": 1, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["eosinophilic"].update({"gaussian_filter_sigma": 0, "median_filter_size": 1, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["epithelial"].update({"gaussian_filter_sigma": 0, "median_filter_size": 1, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["erythrocytes"].update({"gaussian_filter_sigma": 0, "median_filter_size": 1, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})

result_median_filter = convert_from_file(input_filename,
        output_filename = "output/RNAPos Tonsil_20220815-162517-674186_Q001_H&E_filtered.ome.tif",
        # AI_enhancement = False,
        output_pixel_size_x = 0.1721,
        output_pixel_size_y = 0.1721,
        config = config
        )

config["components"]["haematoxylin"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["eosinophilic"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["epithelial"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})
config["components"]["erythrocytes"].update({"gaussian_filter_sigma": 0, "median_filter_size": 0, "sharpen_filter_radius": 0, "sharpen_filter_amount": 0})

result_AI = convert_from_file(input_filename,
        output_filename = "output/RNAPos Tonsil_20220815-162517-674186_Q001_H&E_AI_filter.ome.tif",
        AI_enhancement = True,
        output_pixel_size_x = 0.1721,
        output_pixel_size_y = 0.1721,
        config = config,
        )


result_no_filter = np.transpose(result_no_filter[0], (1,2,0))  # Remove Z dimension and reorder to (H,W,C)
result_median_filter = np.transpose(result_median_filter[0], (1,2,0))  # Remove Z dimension and reorder to (H,W,C)
result_AI = np.transpose(result_AI[0], (1,2,0))  # Remove Z dimension and reorder to (H,W,C)

result_no_filter_flipped = (np.flip(np.flip(result_no_filter, axis=0), axis=1)).astype(np.uint8)
result_median_filter_flipped = (np.flip(np.flip(result_median_filter, axis=0), axis=1)).astype(np.uint8)
result_AI_flipped = (np.flip(np.flip(result_AI, axis=0), axis=1)).astype(np.uint8)


result_image_HE = tifffile.imread("high-parametric-multiplexed-immunofluorescence/8023452/2022_08_10_Tonsil_IF_HE_PostiveIF.ome.tif", level=0)[40500:40500+5810, 28300:28300+5810, :]


# # Create figure with two subplots side by side
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10, 40))

# # Display images
im1 = ax1.imshow(result_no_filter_flipped)
im2 = ax2.imshow(result_AI_flipped)
im3 = ax3.imshow(result_median_filter_flipped)
im4 = ax4.imshow(result_image_HE)

# # Set titles if provided
ax1.set_title("result_no_filter")
ax2.set_title("result_AI")
ax3.set_title("result_median_filter")
ax4.set_title("Real H&E")

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')

# # Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10, 40))

im1 = ax1.imshow(result_no_filter_flipped[3250:3750, 3250:3750, :],
                 interpolation='nearest', resample=False)
im2 = ax2.imshow(result_AI_flipped[3250:3750, 3250:3750, :],
                 interpolation='nearest', resample=False)
im3 = ax3.imshow(result_median_filter_flipped[3250:3750, 3250:3750, :],
                 interpolation='nearest', resample=False)
im4 = ax4.imshow(result_image_HE[3250:3750, 3250:3750, :],
                 interpolation='nearest', resample=False)

for ax, title in zip((ax1, ax2, ax3, ax4),
                     ("result_no_filter", "result_AI", "result_median_filter", "Real H&E")):
    ax.set_title(title)
    ax.axis('off')
    ax.set_aspect('equal')   # ensure square pixels

plt.tight_layout()
plt.show()


In [ ]:
file_path = Path("high-parametric-multiplexed-immunofluorescence/8023452/RNAPos Tonsil_20220815-162517-674186_Q001.ome.tif")
target_channels = ['DNA1', 'DNA2', '106Cd-aSMA', '110Cd-Vim', '143Nd-CD31', '169Tm-Col1', '148Nd-Pan-CK', '158Gd-E-Cad']
rgb_image = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.8)


# Read the TIFF file
image_VHE1 = tifffile.imread("output/2022_08_9_Tonsil_IF_PositiveIF_H&E.ome.tiff", level=0)
image_VHE1 = image_VHE1[23125:23125+2905, 17100:17100+2905, :]

# Create figure with two subplots side by side
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(14, 10))

rgb_image_flipped = (np.flip(np.flip(rgb_image, axis=0), axis=1)*255).astype(np.uint8)


# Display images
im1 = ax1.imshow(rgb_image_flipped)
im2 = ax2.imshow(result_no_filter_flipped)
im3 = ax3.imshow(image_VHE1)
im4 = ax4.imshow(result_image_HE)

print(rgb_image_flipped.shape)
print(result_no_filter_flipped.shape)
print(image_VHE1.shape)
print(result_image_HE.shape)


# Set titles if provided
ax1.set_title("IMC")
ax2.set_title("Virtual H&E from IMC")
ax3.set_title("Virtual H&E from IF")
ax4.set_title("Real H&E")

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 11))
gs = GridSpec(3, 12, figure=fig, height_ratios=[1, 0.84, 0.83])
gs.update(hspace=-0.3)

# Row 1 (3 plots): IF and H&E
ax1 = fig.add_subplot(gs[0, 0:4])
ax2 = fig.add_subplot(gs[0, 4:8])
ax3 = fig.add_subplot(gs[0, 8:12])

# Row 2 (4 plots)
ax4 = fig.add_subplot(gs[1, 0:3])
ax5 = fig.add_subplot(gs[1, 3:6])
ax6 = fig.add_subplot(gs[1, 6:9])
ax7 = fig.add_subplot(gs[1, 9:12])

# Row 3 (4 plots)
ax8 = fig.add_subplot(gs[2, 0:3])
ax9 = fig.add_subplot(gs[2, 3:6])
ax10 = fig.add_subplot(gs[2, 6:9])
ax11 = fig.add_subplot(gs[2, 9:12])

ax1.imshow(image_IF[1200:-700, 800:-500, :][16000:28000, 8000:20000, :])
ax2.imshow(image_HE_IF[1200:-700, 800:-500, :][16000:28000, 8000:20000, :])
ax3.imshow(image_HE[4*1750:4*3250, 4*750:4*2250, :])

# Row 2
ax4.imshow(rgb_image_flipped)
ax5.imshow(result_no_filter_flipped)
ax6.imshow(image_VHE1)
ax7.imshow(result_image_HE)

# Row 3
start_x = 4300
start_y = 3800
size = 400
ax8.imshow(result_no_filter_flipped[start_x:start_x+size, start_y:start_y+size, :], interpolation='nearest', resample=False)
ax9.imshow(result_AI_flipped[start_x:start_x+size, start_y:start_y+size, :], interpolation='nearest', resample=False)
ax10.imshow(result_median_filter_flipped[start_x:start_x+size, start_y:start_y+size, :], interpolation='nearest', resample=False)
ax11.imshow(result_image_HE[start_x:start_x+size, start_y:start_y+size, :], interpolation='nearest', resample=False)

# ----- Titles for remaining panels -----
ax1.set_title("IF")
ax2.set_title("Virtual H&E from IF")
ax3.set_title("Real H&E")

ax4.set_title("IMC")
ax5.set_title("Virtual H&E from IMC")

ax8.set_title("Unfiltered")
ax9.set_title("   Deep learning")
ax10.set_title("Median filter")

# ----- Turn off axes and add borders -----
axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11]
for ax in axes:
    ax.axis('off')
    rect = patches.Rectangle(
        (0, 0), 1, 1, transform=ax.transAxes,
        fill=False, color='black', linewidth=2)
    ax.add_patch(rect)

# ----- Connection patches and zoom rectangles (unchanged references, still valid) -----

# First ConnectionPatch (ax7 <-> ax3)
x0, y0 = 4*1025, 4*775
width, height = 4*375, 4*375
zoom_rect = patches.Rectangle((x0, y0), width, height, linewidth=1, edgecolor='black', facecolor='none')
ax3.add_patch(zoom_rect)

con = ConnectionPatch(xyA=(0, 1), coordsA=ax7.transAxes,
                      xyB=(x0, y0 + height), coordsB=ax3.transData,
                      color="black", lw=1, linestyle='--')
fig.add_artist(con)
con2 = ConnectionPatch(xyA=(1, 1), coordsA=ax7.transAxes,
                       xyB=(x0 + width, y0 + height), coordsB=ax3.transData,
                       color="black", lw=1, linestyle='--')
fig.add_artist(con2)



# Second ConnectionPatch (ax6 <-> ax2)
x0, y0 = 4*2075, 4*1450
width, height = 4*750, 4*750
zoom_rect = patches.Rectangle((x0, y0), width, height, linewidth=1, edgecolor='black', facecolor='none')
ax2.add_patch(zoom_rect)

con = ConnectionPatch(xyA=(0, 1), coordsA=ax6.transAxes,
                      xyB=(x0, y0 + height), coordsB=ax2.transData,
                      color="black", lw=1, linestyle='--')
fig.add_artist(con)
con2 = ConnectionPatch(xyA=(1, 1), coordsA=ax6.transAxes,
                       xyB=(x0 + width, y0 + height), coordsB=ax2.transData,
                       color="black", lw=1, linestyle='--')
fig.add_artist(con2)




# Third ConnectionPatch (ax11 <-> ax7)
x0, y0 = start_y, start_x
width, height = size, size
zoom_rect = patches.Rectangle((x0, y0), width, height, linewidth=1, edgecolor='black', facecolor='none')
ax7.add_patch(zoom_rect)

con = ConnectionPatch(xyA=(0, 1), coordsA=ax11.transAxes,
                      xyB=(x0, y0 + height), coordsB=ax7.transData,
                      color="black", lw=1, linestyle='--')
fig.add_artist(con)
con2 = ConnectionPatch(xyA=(1, 1), coordsA=ax11.transAxes,
                       xyB=(x0 + width, y0 + height), coordsB=ax7.transData,
                       color="black", lw=1, linestyle='--')
fig.add_artist(con2)




# Fourth ConnectionPatch (ax9 <-> ax5)
x0, y0 = start_y, start_x
width, height = size, size
zoom_rect = patches.Rectangle((x0, y0), width, height, linewidth=1, edgecolor='black', facecolor='none')
ax5.add_patch(zoom_rect)

con = ConnectionPatch(xyA=(0, 1), coordsA=ax9.transAxes,
                      xyB=(x0, y0 + height), coordsB=ax5.transData,
                      color="black", lw=1, linestyle='--')
fig.add_artist(con)
con2 = ConnectionPatch(xyA=(1, 1), coordsA=ax9.transAxes,
                       xyB=(x0 + width, y0 + height), coordsB=ax5.transData,
                       color="black", lw=1, linestyle='--')
fig.add_artist(con2)



# ===== Add scale bars to the remaining axes =====
fig.canvas.draw()  # Ensure renderer is ready

add_scalebar(ax1, pixel_size=0.6882/2, scalebar_length=1, length_unit='mm', bar_color='white', text_color='white')
add_scalebar(ax4, pixel_size=1, scalebar_length=500, length_unit='um', bar_color='white', text_color='white')
add_scalebar(ax8, pixel_size=0.1721, scalebar_length=25, length_unit='um', bar_color='white', text_color='white')

plt.savefig('figures/examples.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()
